In [ ]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch.exceptions import RequestError
import pandas as pd


In [4]:
df = pd.read_parquet("../data/df.parquet")


In [5]:
df.head(10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047,United Kingdom


In [6]:
df.shape


(397924, 8)

In [2]:
es = Elasticsearch({"scheme": "http", "host": "host.docker.internal", "port": 9200},  max_retries=3,
                   retry_on_timeout=True, request_timeout=3)


In [3]:
if not es.ping():
    raise ValueError("Connection failed")


In [7]:
def df_to_es(df, es_index):
    for df_idx, line in df.iterrows():
        yield {
            "_index": es_index,
            "_id": df_idx,
            "_source": {
                "InvoiceNo": line[0],
                "StockCode": line[1],
                "Description": line[2],
                "Quantity": line[3],
                "InvoiceDate": line[4],
                "UnitPrice": line[5],
                "CustomerID": line[6],
                "Country": line[7]

            }
        }


In [8]:
%%time
helpers.bulk(es, df_to_es(df, "transactions"), raise_on_error=False)


CPU times: total: 45.2 s
Wall time: 1min 14s


(397924, [])

In [9]:
query = {
    "query": {
        "match": {
            "InvoiceNo": {
                "query": '536366'
            }
        }
    }
}

res = es.search(index="transactions", body=query)


C:\Users\silverstone\AppData\Local\Temp\ipykernel_12016\3582670259.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="transactions", body=query)


In [10]:
res["hits"]["hits"]


[{'_index': 'transactions',
  '_id': '7',
  '_score': 11.974142,
  '_source': {'InvoiceNo': '536366',
   'StockCode': '22633',
   'Description': 'HAND WARMER UNION JACK',
   'Quantity': 6,
   'InvoiceDate': '12/1/2010 8:28',
   'UnitPrice': 1.85,
   'CustomerID': '17850',
   'Country': 'United Kingdom'}},
 {'_index': 'transactions',
  '_id': '8',
  '_score': 11.974142,
  '_source': {'InvoiceNo': '536366',
   'StockCode': '22632',
   'Description': 'HAND WARMER RED POLKA DOT',
   'Quantity': 6,
   'InvoiceDate': '12/1/2010 8:28',
   'UnitPrice': 1.85,
   'CustomerID': '17850',
   'Country': 'United Kingdom'}}]